# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>
<hr>

_Embeddings_ are representation of words thanks to vectors. These embeddings can be learnt within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which the ÌMDB dataset: it corresponds to sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=25)

2022-05-17 22:24:46.622428: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 22:24:46.622579: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-17 22:24:49.718635: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-17 22:24:49.718717: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-17 22:24:49.718746: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-O26C6N05): /proc/driver/nvidia/version does not exist
2022-05-17 22:24:49.719371: I tensorflow/core/platform/cpu_fe

Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In the previous exercise, we jointly learnt a representation for the words, and feed this representation to a RNN, as shown here : 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which can slow the convergence, and make it harder!

For that reason, we will separate the steps of learning the word representation and feeding it to a RNN. As shown here : 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with the word2vec.

The drawback is indeed that the learnt embedding is not _specifically_ designed for our task. However, learning it independently of the task at hand (sentiment analysis) has some advantages : 
- it is very fast to do in general (with word2vec)
- the representation learnt by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As on this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`.

In [2]:
X_train[0]

['this',
 'was',
 'an',
 'absolutely',
 'terrible',
 'movie',
 "don't",
 'be',
 'lured',
 'in',
 'by',
 'christopher',
 'walken',
 'or',
 'michael',
 'ironside',
 'both',
 'are',
 'great',
 'actors',
 'but',
 'this',
 'must',
 'simply',
 'be',
 'their',
 'worst',
 'role',
 'in',
 'history',
 'even',
 'their',
 'great',
 'acting',
 'could',
 'not',
 'redeem',
 'this',
 "movie's",
 'ridiculous',
 'storyline',
 'this',
 'movie',
 'is',
 'an',
 'early',
 'nineties',
 'us',
 'propaganda',
 'piece',
 'the',
 'most',
 'pathetic',
 'scenes',
 'were',
 'those',
 'when',
 'the',
 'columbian',
 'rebels',
 'were',
 'making',
 'their',
 'cases',
 'for',
 'revolutions',
 'maria',
 'conchita',
 'alonso',
 'appeared',
 'phony',
 'and',
 'her',
 'pseudo',
 'love',
 'affair',
 'with',
 'walken',
 'was',
 'nothing',
 'but',
 'a',
 'pathetic',
 'emotional',
 'plug',
 'in',
 'a',
 'movie',
 'that',
 'was',
 'devoid',
 'of',
 'any',
 'real',
 'meaning',
 'i',
 'am',
 'disappointed',
 'that',
 'there',
 'are

In [3]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

Let's look at the embedded representation of some words.

You can use `word2vec.wv` as a dictionary.
For instance, `word2vec.wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation were not learn). 

In [4]:
word2vec.wv['dog']

array([-0.258369  ,  0.527599  ,  0.03604262,  0.3795117 , -0.38310987,
       -0.61322343,  0.25070885,  0.53591174,  0.07278352,  0.08035191,
       -0.12545843, -0.20492831,  0.14453278,  0.18144794, -0.13063766,
       -0.00301276, -0.2760755 , -0.35323864, -0.3926505 , -0.27271563,
        0.32195383,  0.27741188,  0.21443802, -0.28214633, -0.16205457,
        0.1435409 , -0.1134486 , -0.04891549, -0.13883305,  0.34043097,
        0.12210411,  0.20989661,  0.17003274, -0.18149996, -0.24811317,
        0.44568616, -0.087736  ,  0.00832892, -0.03468119, -0.33354577,
        0.28613514, -0.3701918 , -0.5286877 , -0.00218877,  0.18039724,
        0.08696224,  0.01455728, -0.13345902,  0.01065398, -0.06056111,
        0.12983453, -0.12504534, -0.22227743, -0.32725817, -0.10477861,
        0.04189412,  0.05760623, -0.22473007, -0.17862347,  0.29292077,
        0.05251765,  0.04747274, -0.35177213,  0.06025459,  0.01646016,
        0.2975085 ,  0.05631257,  0.5031291 , -0.56373227,  0.29

In [5]:
word2vec.wv['film']

array([ 1.7574311 , -0.8559548 ,  0.6140554 , -0.50663245,  2.842134  ,
       -0.59958386, -0.4835693 ,  0.68827957, -0.31597677, -0.141371  ,
       -0.3970512 , -1.8233407 ,  0.5756624 ,  1.9833033 , -0.7898381 ,
       -0.24651788, -0.7289643 ,  0.96204054, -1.2038152 , -0.5852995 ,
        0.63902426,  0.5851941 ,  1.1576966 ,  0.7705997 ,  0.31601748,
       -0.27763748, -1.8463669 , -0.37295735,  0.89515585, -1.4704354 ,
        1.9848735 ,  0.02103379,  0.07201786, -1.2185012 ,  0.30194154,
       -0.88175225,  1.117591  ,  0.6681689 , -0.22643574, -2.3323953 ,
       -1.4324025 , -0.37185812,  0.08674255, -0.27101707, -1.4737467 ,
        1.4404858 , -0.52350754, -0.22793233,  2.7982562 ,  0.16937312,
       -0.57113785, -1.6889006 , -0.29799676, -0.54761326,  1.4799191 ,
        0.30412775,  0.6871216 , -1.0445278 , -1.2593294 , -0.4368979 ,
       -0.23842987,  1.0744798 ,  2.019303  ,  1.3888499 , -0.86033183,
        1.4890307 , -0.2622349 ,  1.2819302 , -0.36478618,  0.58

In [6]:
word2vec.wv['blob']

array([-0.00796493,  0.21472812, -0.04704149,  0.02278395, -0.07922689,
       -0.28558916,  0.03039944,  0.3157192 , -0.1195132 , -0.11033669,
       -0.01934515, -0.0975147 ,  0.03337905,  0.11707298,  0.03488785,
        0.01259572,  0.06469938, -0.05556417, -0.10432813, -0.2794854 ,
        0.12279962, -0.04819744,  0.16643408, -0.12454268, -0.08090362,
        0.09160627, -0.11060938,  0.0518852 , -0.26201212, -0.05854265,
        0.17934753,  0.01836973,  0.00636388, -0.2920246 , -0.03550041,
        0.0939504 ,  0.06457137, -0.17085439, -0.09433189, -0.16746028,
       -0.06245759, -0.27105352, -0.06111815, -0.04091402,  0.22161232,
       -0.00818355, -0.09773099, -0.14187656,  0.03796668,  0.06737788,
        0.0011893 , -0.0518213 , -0.11166473, -0.0932867 ,  0.03517812,
        0.1416269 ,  0.19138332,  0.02858501, -0.07090294,  0.04123184,
        0.11352063, -0.10270803, -0.03833651,  0.02333856, -0.12478112,
        0.26960298, -0.05925027,  0.167136  , -0.09972696,  0.22

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [7]:
len(word2vec.wv['dog'])

100

How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the `word2vec.most_similar(...)` method that, given an input word, display the "closest" words in the embedding space. If the embedding is well done, then words that have close meanings will have close representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [8]:
word2vec.wv.most_similar('woman')

[('girl', 0.9314076900482178),
 ('man', 0.8920845985412598),
 ('lady', 0.8423522114753723),
 ('doctor', 0.8368729948997498),
 ('boy', 0.8137881755828857),
 ('cop', 0.8004390597343445),
 ('child', 0.7805746793746948),
 ('rich', 0.7709635496139526),
 ('guy', 0.7690020799636841),
 ('scientist', 0.7642442584037781)]

In [9]:
word2vec.wv.most_similar('director')

[('writer', 0.7702202200889587),
 ('actor', 0.6687769889831543),
 ('filmmaker', 0.6362221240997314),
 ('screenplay', 0.6185498237609863),
 ('actress', 0.6116209626197815),
 ('attenborough', 0.604540228843689),
 ('producer', 0.603386640548706),
 ('role', 0.6033696532249451),
 ('casting', 0.6018241047859192),
 ('mckay', 0.5939505696296692)]

In [10]:
word2vec.wv.most_similar('batman')

[('trilogy', 0.8223293423652649),
 ('columbo', 0.8029468059539795),
 ('classics', 0.8010308146476746),
 ('eighties', 0.8005201816558838),
 ("'the", 0.7913722991943359),
 ('scanners', 0.79061359167099),
 ('trek', 0.7855579853057861),
 ('1969', 0.7832911610603333),
 ('iii', 0.7802385091781616),
 ('cartoon', 0.7793601155281067)]

Similarly to `most_similar` used on words directly, we can use `similar_by_vector` on vectors to do the same thing :

In [11]:
word2vec.wv.similar_by_vector(word2vec.wv['cat'])

[('cat', 0.9999999403953552),
 ('assistant', 0.9242337346076965),
 ('owner', 0.9219779968261719),
 ('apartment', 0.9207000136375427),
 ('sons', 0.9184197187423706),
 ('boss', 0.9087929725646973),
 ('chasing', 0.9055609107017517),
 ('uncle', 0.9033519625663757),
 ('knife', 0.9023448824882507),
 ('chief', 0.9013227224349976)]

# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [12]:
word2vec.wv['good'] - word2vec.wv['bad']

array([-0.16043146, -0.02155775,  0.29767385,  0.29724646,  0.42517292,
        0.05608812, -0.8561748 ,  0.12123728,  0.01042698, -0.04395425,
        0.1470415 ,  1.5790956 ,  1.3118674 ,  0.6737417 ,  0.679517  ,
        1.1118333 , -0.05650565,  0.8100716 ,  0.09739177, -0.39247513,
       -0.5306072 ,  0.4731603 , -0.7037029 , -0.89233017, -0.44619775,
        0.6150319 , -0.5991274 ,  1.3675001 , -1.2469273 ,  0.08916992,
       -0.56459326,  0.26689553, -0.8212335 ,  1.2166818 , -0.80098397,
       -1.7406461 , -0.31929773,  0.5206584 ,  0.06735379,  1.3525926 ,
       -0.227873  , -1.1845406 , -0.37686276, -0.19984174,  0.20419121,
        0.34527874,  1.2652855 ,  0.8067384 , -1.1874007 ,  0.93756866,
        0.12166911, -0.14435321, -0.4908343 , -1.2964427 ,  0.33789712,
       -0.01434094, -1.180672  ,  0.95501024, -0.60325456, -0.5214343 ,
        1.0920053 ,  0.11604524,  0.05502705,  0.98324776,  0.45715237,
        0.12535971,  0.13587785,  1.42179   ,  0.02111667, -0.04

Now, image for a second that, somehow, the following equality holds true - just for a second

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to 

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be foolish of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [13]:
res = word2vec.wv['good'] - word2vec.wv['bad'] + word2vec.wv['stupid']

In [14]:
res

array([-0.4843366 , -0.222885  ,  0.45190048, -0.15867293,  0.783609  ,
        0.10416564, -1.5881343 , -0.5500685 , -0.3781805 , -0.13951948,
        0.5683843 ,  0.9546979 ,  1.6554816 ,  0.70661825,  1.166911  ,
        0.6465915 ,  0.61440873, -0.24012697,  0.64357114, -1.7967167 ,
        0.49305212,  0.46612537, -0.44967556, -0.35886377, -0.50792056,
       -0.17999375,  0.01190227,  0.3041495 , -1.2956668 , -0.10132815,
        0.1328221 ,  0.7375649 , -1.034113  ,  0.14215648, -1.2526181 ,
       -1.1318672 , -1.4059184 ,  0.9960068 , -0.21282125, -0.22467172,
        0.0982669 , -0.9754416 , -0.6856043 , -0.18426338, -0.16865021,
        0.8154522 ,  0.09041166,  0.50977904, -1.117307  ,  1.0927756 ,
        1.2162352 , -0.4618031 , -0.26986742, -0.60478586,  0.33795345,
       -0.19009534, -0.7435099 ,  1.2899994 , -1.4216609 , -0.688429  ,
        0.7250619 , -0.07383861,  0.321989  ,  0.8444496 , -0.02270222,
        0.65694445,  0.6740686 ,  1.3529897 , -0.29003105,  0.46

We earlier said that, for any vector, it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `word2vec.wv.similar_by_vector` function) of `res`

In [15]:
word2vec.wv.similar_by_vector(res)

[('good', 0.7284103035926819),
 ('nice', 0.7179179787635803),
 ('decent', 0.6599467396736145),
 ('fine', 0.6243587136268616),
 ('interesting', 0.5955530405044556),
 ('believable', 0.5952949523925781),
 ('great', 0.5915830135345459),
 ('cute', 0.5890123844146729),
 ('cool', 0.5870103240013123),
 ('funny', 0.5781408548355103)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [16]:
res = word2vec.wv['queen'] - word2vec.wv['king'] + word2vec.wv['actor']
word2vec.wv.similar_by_vector(res)

[('actor', 0.9722597599029541),
 ('actress', 0.9190306067466736),
 ('role', 0.8136066794395447),
 ('performance', 0.7571848630905151),
 ('villain', 0.7459296584129333),
 ('oscar', 0.7334705591201782),
 ('accent', 0.704143226146698),
 ('genius', 0.6983972787857056),
 ('job', 0.6914918422698975),
 ('acquaintance', 0.6859427690505981)]

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)` , actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it chooses many splits in the different sentences, choose some words as inputs $X$  and a word as output $y$ which it tries to predict, in the embedding space.

And as any neural network, Word2Vec has some hyperparameters. Let's check some. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [17]:
word2vec_2 = Word2Vec(sentences=X_train, vector_size=50)
len(word2vec_2.wv['movie'])

50

❓ **Question** ❓ Use the `word2vec.wv.key_to_index` attribute to display the size of the learnt vocabulary. On the other hand, compare it to the number of different words in `X_train`.

In [18]:
len(word2vec_2.wv.key_to_index)

14416

In [19]:
# words = []
# for inside_list in X_train:
#     for word in inside_list:
#         words.append(word)
        
# unique_words = set(words)

# len(unique_words)

In [20]:
len(set([word for inside_list in X_train for word in inside_list]))

47215

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been train on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec :  `min_count`. 

`min_count` is a integer that tells you how many occurences a given word should have to be learn in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vec that you have trained (you can choose any `vector_size` you want).

In [21]:
word2vec_3 = Word2Vec(sentences=X_train, vector_size=50, min_count=10)
len(word2vec_3.wv.key_to_index)

8817

In [22]:
word2vec_4 = Word2Vec(sentences=X_train, vector_size=50, min_count=3)
len(word2vec_4.wv.key_to_index)

20607

Remember that we say that word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Learn a new `word2vec_5` model with a `window` different than previously (default is 5).

In [24]:
word2vec_5 = Word2Vec(sentences=X_train, vector_size=50, min_count=10, window=10)

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the one that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here :

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [25]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
    return np.array(embedded_sentence)
    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentence (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [26]:
def embedding(word2vec, sentences):
    embedded_sentences = []
    for sentence in sentences:
        embedded_sentences.append(embed_sentence(word2vec, sentence))
    return embedded_sentences
    
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad them in order to have tensors that can be divided in batch sizes during the optimization. Store the padedd values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [28]:
X_train_pad = pad_sequences(X_train, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post')

In [29]:
assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)